In [3]:
!pip install transformers torch

  Using cached torch-2.0.0-cp39-none-macosx_11_0_arm64.whl (55.8 MB)
  Using cached sympy-1.11.1-py3-none-any.whl (6.5 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached MarkupSafe-2.1.2-cp39-cp39-macosx_10_9_universal2.whl (17 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [1]:
!ls

1000_ama_posts.csv
10_IAmA_posts.csv
20_IAmA_posts.csv
all_1000_IAmA_posts.csv
all_1000_IAmA_posts_with_wiki.csv
augment_input.ipynb
context_processing.ipynb
data.csv
data_2014.csv
data_2015_to_2021.csv
data_2015_to_2021_wiki.csv
data_2015_to_2021_wiki_summary.csv
data_2015_to_2021_wiki_summary_clean.csv
data_2015_to_2021_wiki_summary_title_clean.csv
data_2015_to_2021_with_cherry_picked_comments.csv
data_2015_to_2021_with_context.csv
data_2015_to_2021_with_topics.csv
enwiki-latest-all-titles-in-ns0.gz
final_582_data_collection.ipynb
make_dataset.ipynb
openai-api.ipynb
output_dir
pmaw.ipynb
qa_evaluator.csv
qa_evaluator_train.csv
qa_evaluator_val.csv
question_generator.csv
question_generator_train.csv
question_generator_val.csv
report.aux
report.fdb_latexmk
report.fls
report.log
report.out
report.pdf
report.synctex.gz
report.tex
top_10_ama_posts.csv
top_10_ama_posts.json
toy_t5
toy_training_data.csv
train-qt-eval.ipynb
train.ipynb
train_GPT2.ipynb


In [30]:
import pandas as pd

In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-base-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

/Users/ehsanulkabir/miniconda/envs/final582/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
example = "I’m Bill Gates, co-chair of the Bill & Melinda Gates Foundation. Ask Me Anything."

ner_results = nlp(example)
print(ner_results)

# filter ner results to only include PERSON entities
person_entities = [entity for entity in ner_results if entity['entity'] == 'I-PER' or entity['entity'] == 'B-PER']
# get 'word' key from each entity
person_entities = [entity['word'] for entity in person_entities]


[{'entity': 'B-PER', 'score': 0.99956685, 'index': 4, 'word': 'Bill', 'start': 4, 'end': 8}, {'entity': 'I-PER', 'score': 0.9995931, 'index': 5, 'word': 'Gates', 'start': 9, 'end': 14}, {'entity': 'B-ORG', 'score': 0.99704903, 'index': 12, 'word': 'Bill', 'start': 32, 'end': 36}, {'entity': 'I-ORG', 'score': 0.9982393, 'index': 13, 'word': '&', 'start': 37, 'end': 38}, {'entity': 'I-ORG', 'score': 0.9988324, 'index': 14, 'word': 'Melinda', 'start': 39, 'end': 46}, {'entity': 'I-ORG', 'score': 0.9976398, 'index': 15, 'word': 'Gates', 'start': 47, 'end': 52}, {'entity': 'I-ORG', 'score': 0.9944133, 'index': 16, 'word': 'Foundation', 'start': 53, 'end': 63}]


In [1]:
' '.join(person_entities)

NameError: name 'person_entities' is not defined

In [23]:
filename = '20_IAmA_posts.csv'
df = pd.read_csv(filename)
# df = pd.read_json(filename)

for index, row in df.iterrows():
    ner_results = nlp(row['title'])
    # body = row['body']
    # if type(body) != str or len(body) < 10:
    #     continue
    # ner_results = nlp(body)
    person_entities = [entity for entity in ner_results if entity['entity'] == 'I-PER' or entity['entity'] == 'B-PER']
    person_entities = [entity['word'] for entity in person_entities]
    df.at[index, 'person_entities'] = ' '.join(person_entities)

In [32]:
import gzip

# Load the Wikipedia article titles into a set for efficient lookup
wiki_titles = set()
with gzip.open("enwiki-latest-all-titles-in-ns0.gz", "rt", encoding="utf-8") as f:
    for title in f:
        title = title.strip()
        title = title.replace("_", " ")
        wiki_titles.add(title.strip())

# Check if a person name has a Wikipedia article associated with it
person_name = "Bill Clinton"
if person_name in wiki_titles:
    print(f"{person_name} has a Wikipedia article!")
else:
    print(f"{person_name} does not have a Wikipedia article.")

Bill Clinton has a Wikipedia article!


In [31]:
title

'\U0010ffff\n'

In [30]:
wiki_titles = list(wiki_titles)
wiki_titles

TypeError: 'set' object is not subscriptable

In [2]:
from transformers import pipeline, set_seed, AutoTokenizer, AutoModelWithLMHead
generator = pipeline('text-generation', model='gpt2')
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [3]:
# Define the input text
ama_post = "I am a neuroscientist studying the brain. I'm interested in the relationship between the brain and behavior."

In [26]:
def get_profession(ama_post):
    if type(ama_post) != str or len(ama_post) < 10:
        return ''
    elif len(ama_post) > 1000:
        ama_post = ama_post[:1000]
    set_seed(42)
    query_string = f"{ama_post}. I worked as a"
    # count the number of tokens in the query string
    num_tokens = len(tokenizer.encode(query_string))
    output = generator(query_string, max_length=num_tokens+5, num_return_sequences=1)[0]['generated_text']
    # print(output)
    output = output.replace(query_string, '')
    output = output[1:].split(' ')[0]
    # print(output)
    return output

In [44]:
filename = 'all_1000_IAmA_posts_with_wiki.csv'
df = pd.read_csv(filename)

In [45]:
df.iloc[-2]["comments"]

'[\'Have you ever had word come down to report with a bias or to omit facts in order to push an agenda? Had an article edited or denied due to the information not swaying a certain way? Or have you heard of other who have?  \', "I don\'t know much about you and haven\'t seen much Vice news, but this seems like a good place to ask, what is happening in the world of journalism? I mean specifically speaking, could you shed some light on what your world is dealing with right now and how your able to maintain this daily life? \\n\\nMuch thanks, and glad you made it out of D.C. okay. ", \'[deleted]\', \'Why is Vice so shit nowadays? They flood Facebook with bullshit articles.\', "What\'s your relationship with Gavin Mcinnes like?", "How do you feel about comedy/satire being a huge part of modern journalism? My theory is that you can\'t fake a laugh because you need honesty in some form for comedy.", \'Did you expect to get out? Have you, or colleagues, been arrested in a situation like that 

In [37]:
import ast

In [57]:
ast.literal_eval(df.iloc[49]["comments"])[6]

"I loved reading, er, listening to your new book in audiobook format. When I was finished, I was saddened to find that this is the only book you've ever narrated. Any plans to narrate more books in your awesome voice? What genre of book besides autobiography would you be interested in doing?"

In [64]:
df["comments"] = df["comments"].apply(lambda x: ast.literal_eval(x))

In [87]:
# df['input_text'] = f'<OP_NAME> {df["person_entities"]} <OP_TITLE> {df["title"]} <OP_POST> {df["body"]} <OP_BIO> {df["wiki_summary"]}'
# df['input_text'] = df.apply(lambda row: f'<OP_NAME> {row["person_entities"]} [SEP] <OP_TITLE> {row["title"]} [SEP] <OP_POST> {row["body"]} [SEP] <OP_BIO> {row["wiki_summary"]}', axis=1)
# df['input_text'] = df.apply(lambda row: f'<OP_NAME> {row["person_entities"]} </s> <OP_TITLE> {row["title"]} </s> <OP_POST> {row["body"]} </s> <OP_BIO> {row["wiki_summary"]} </s> Questions from the Reddit:', axis=1)
df['input_text'] = df['person_entities'].apply(lambda x: f"Some questions Reddit users might have to {x} are:")


In [86]:
df.iloc[13]["wiki_summary"]

"Adam Whitney Savage (born July 15, 1967) is an American special effects designer and fabricator, actor, educator, and television personality and producer, best known as the former co-host (with Jamie Hyneman) of the Discovery Channel television series MythBusters and Unchained Reaction. His model work has appeared in major films, including Star Wars: Episode II – Attack of the Clones and The Matrix Reloaded. He is the host of the TV program Savage Builds, which premiered on the Science Channel on June 14, 2019. He is most active on the platform Adam Savage's Tested which includes a website and a YouTube channel."

In [88]:
# from the list of comments filter out the ones that are too short or too long or doesn't contain a question mark
# replace \n with space
# keep the first 20 comments
# join the comments with \n
df['output_text'] = df['comments'].apply(lambda x: '\n'.join([comment.replace('\n', ' ') for comment in x if len(comment) > 10 and len(comment) < 1000 and '?' in comment][:20]))

In [89]:
df[['input_text', 'output_text']].to_csv('toy_training_data.csv', index=False)

In [27]:
df['profession'] = df['body'].apply(get_profession)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [29]:
!pip install transformers deepspeed

import torch
from transformers import GPTJForCausalLM, GPT2TokenizerFast
from transformers import pipeline

# Load the pre-trained GPT-J model and tokenizer
model_name = "EleutherAI/gpt-j-6B"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPTJForCausalLM.from_pretrained(model_name)

# Define the input text
ama_post = "I am a neuroscientist studying the brain. I'm interested in the relationship between the brain and behavior."

# Preprocess the input text
text = ama_post.lower()  # Convert text to lowercase
text = re.sub(r'<[^>]+>', '', text)  # Remove HTML tags
text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stop words

# Tokenize the preprocessed text
input_ids = tokenizer(text, return_tensors="pt").input_ids

# Use the GPT-J model to generate entity labels for each token in the input text
with torch.no_grad():
    outputs = model(input_ids)

generated_labels = torch.argmax(outputs.logits, dim=2).squeeze()

# Extract the professions from the input text
professions = []
for i, label in enumerate(generated_labels):
    if label == 1:  # "1" represents "B-PRO" (beginning of a profession)
        profession_tokens = [input_ids[0][i].item()]
        for j in range(i+1, len(generated_labels)):
            if generated_labels[j] == 2:  # "2" represents "I-PRO" (inside a profession)
                profession_tokens.append(input_ids[0][j].item())
            else:
                break
        profession = tokenizer.decode(profession_tokens)
        professions.append(profession)

# Print the extracted professions
print("Professions found in the Reddit AskMeAnything post: ", professions)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.2/766.2 kB 7.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Using cached hjson-3.1.0-py3-none-any.whl (54 kB)
  Using cached ninja-1.11.1-py2.py3-none-macosx_10_9_universal2.macosx_10_9_x86_64.macosx_11_0_arm64.macosx_11_0_universal2.whl (270 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl (22 kB)
  Using cached pydantic-1.10.7-cp39-cp39-macosx_11_0_arm64.whl (2.6 MB)
  Created wheel for deepspeed: filename=deepspeed-0.9.1-py3-none-any.whl size=798536 sha256=7256cf2584db692370fbb3f3fa7b5410aac67a990cc725a32629ecc70a8e25e6
  Stored in directory: /Users/ehsanulkabir/Library/Caches/pip/wheels/7

KeyboardInterrupt: 

In [90]:
!pip install nltk

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)


In [91]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter

nltk.download('punkt')
nltk.download('stopwords')

def extract_keywords(sentence):
    # Tokenize the sentence into words
    words = word_tokenize(sentence)
    
    # Remove stop words from the words list
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word.lower() not in stop_words]
    
    # Count the frequency of each word in the words list
    word_freq = Counter(words)
    
    # Return the most common word
    return word_freq.most_common(1)[0][0]


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ehsanulkabir/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ehsanulkabir/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [94]:
sentence = "How did you start your journey of entrepreneurship?"
keyword = extract_keywords(sentence)
print(keyword) # Output: "cat"

start
